In [1]:
using LinearAlgebra, BenchmarkTools

# 行列の列jを、ベクトルの第j要素で割る演算の書き方比較

In [2]:
dim = 1000

1000

In [3]:
A = randn(dim, dim)
b = randn(dim)

1000-element Array{Float64,1}:
  1.7467975614604583
  2.8648690518870015
 -1.1765828159427782
  0.20007092686238814
  0.6195680312412676
 -0.35776044630612097
 -0.815113492741773
  0.951571747967914
  0.2576862582308946
  1.269649708717053
  1.5094889827992033
  2.017835903946617
 -0.5910780096488174
  ⋮
  0.717630835260704
  0.038905752823954025
 -0.20312888352220804
 -1.9521317520932593
  1.2595694397992767
 -0.47462544255187095
  0.7878719226400558
  0.13296870965694127
 -0.4439373114452578
 -0.06791664520168086
 -1.1752193493596637
  0.9915225848553507

## Referenceになる愚直に計算して正解を作る関数を用意

In [4]:
#正解行列を作る
function calc1(A::Array{Float64,2}, b::Vector{Float64})
    C = similar(A)
    for j in 1:dim
        C[:,j] .= A[:,j] ./ b[j]
    end
    return C
end

calc1 (generic function with 1 method)

In [5]:
C = calc1(A, b)

1000×1000 Array{Float64,2}:
  0.150465    0.669375   -0.634225    …   10.4756     0.945072    1.31976
 -0.608588    0.583223    0.779926        28.8508    -0.96918     1.03514
 -1.57155    -0.23582    -0.0720551       -8.54521    0.529165    0.478813
  0.283903    0.286903    0.649434        -9.20486    0.0825655   1.07761
 -0.338007    0.200325    0.0964925       10.4225    -1.35518     0.190789
 -1.09729    -0.25865     0.940404    …   10.6477     0.62146    -0.0380101
  0.898136    0.174081    0.469181       -22.4483    -0.748625   -0.707328
  0.828013   -0.224381    0.573171       -12.7664     1.17309    -0.51876
 -0.76583    -0.32225    -0.595427        -7.43186   -0.810677   -0.242962
 -0.826892   -0.156963   -0.943456       -12.6295    -0.0534637  -0.989267
  0.0322462   0.0256327   1.15996     …  -11.0683     0.741776    0.519832
  0.143645    0.0254965  -0.913508       -16.8898    -0.411393    0.0781426
 -0.301467   -0.0380585   0.447166        10.1023     0.172388   -2.50261


In [6]:
@benchmark C = calc1(A, b)

BenchmarkTools.Trial: 
  memory estimate:  15.71 MiB
  allocs estimate:  17448
  --------------
  minimum time:     10.108 ms (0.00% GC)
  median time:      12.252 ms (0.00% GC)
  mean time:        14.763 ms (16.68% GC)
  maximum time:     27.810 ms (38.20% GC)
  --------------
  samples:          339
  evals/sample:     1

## 引数破壊版

In [7]:
function calc2!(C::Array{Float64,2}, A::Array{Float64,2}, b::Vector{Float64})
    for j in 1:dim
        C[:,j] .= A[:,j] ./ b[j]
    end
    return C
end

calc2! (generic function with 1 method)

In [8]:
@benchmark calc2!(C, A, b)

BenchmarkTools.Trial: 
  memory estimate:  8.08 MiB
  allocs estimate:  17446
  --------------
  minimum time:     4.435 ms (0.00% GC)
  median time:      5.390 ms (0.00% GC)
  mean time:        6.295 ms (8.02% GC)
  maximum time:     23.224 ms (60.49% GC)
  --------------
  samples:          795
  evals/sample:     1

引数破壊版は2倍弱速い

## Eq1. A * inv(Diagonal(b))

In [9]:
max(abs.(C - A * inv(Diagonal(b)))...)

4.547473508864641e-13

In [10]:
@benchmark A * inv(Diagonal(b))

BenchmarkTools.Trial: 
  memory estimate:  7.64 MiB
  allocs estimate:  7
  --------------
  minimum time:     5.115 ms (0.00% GC)
  median time:      5.606 ms (0.00% GC)
  mean time:        6.927 ms (14.46% GC)
  maximum time:     19.060 ms (48.23% GC)
  --------------
  samples:          720
  evals/sample:     1

## Eq2. A * Diagonal(1 ./ b)

In [11]:
max(abs.(C - A * Diagonal(1 ./ b))...)

4.547473508864641e-13

In [12]:
@benchmark A * Diagonal(1.0 ./ b)

BenchmarkTools.Trial: 
  memory estimate:  7.64 MiB
  allocs estimate:  8
  --------------
  minimum time:     5.191 ms (0.00% GC)
  median time:      5.664 ms (0.00% GC)
  mean time:        6.991 ms (14.56% GC)
  maximum time:     19.668 ms (60.71% GC)
  --------------
  samples:          715
  evals/sample:     1

**Eq1.の方が速い**

## .演算子で結果を他の配列に代入してみる（配列確保されずに速くなる？）

## Eq3. A2 .= A * inv(Diagonal(b))

In [13]:
A2 = similar(A)
@benchmark A2 .= A * inv(Diagonal(b))

BenchmarkTools.Trial: 
  memory estimate:  7.64 MiB
  allocs estimate:  9
  --------------
  minimum time:     6.872 ms (0.00% GC)
  median time:      7.537 ms (0.00% GC)
  mean time:        8.897 ms (11.39% GC)
  maximum time:     19.881 ms (38.95% GC)
  --------------
  samples:          561
  evals/sample:     1

かえって遅い？ **右辺の演算の際に配列は確保されて、それをさらにコピーしている**っぽい挙動。

## Eq4. mul!(A2, A, inv(Diagonal(b)))

In [14]:
mul!(A2, A, inv(Diagonal(b)))
max(abs.(C - A2)...)

4.547473508864641e-13

In [15]:
@benchmark mul!(A2, A, inv(Diagonal(b)))

BenchmarkTools.Trial: 
  memory estimate:  8.06 KiB
  allocs estimate:  5
  --------------
  minimum time:     1.125 ms (0.00% GC)
  median time:      1.216 ms (0.00% GC)
  mean time:        1.318 ms (0.00% GC)
  maximum time:     11.814 ms (0.00% GC)
  --------------
  samples:          3751
  evals/sample:     1

**↑最速**

A自身に解を代入してみる

In [16]:
mul!(A, A, inv(Diagonal(b)))

1000×1000 Array{Float64,2}:
  0.150465    0.669375   -0.634225    …   10.4756     0.945072    1.31976
 -0.608588    0.583223    0.779926        28.8508    -0.96918     1.03514
 -1.57155    -0.23582    -0.0720551       -8.54521    0.529165    0.478813
  0.283903    0.286903    0.649434        -9.20486    0.0825655   1.07761
 -0.338007    0.200325    0.0964925       10.4225    -1.35518     0.190789
 -1.09729    -0.25865     0.940404    …   10.6477     0.62146    -0.0380101
  0.898136    0.174081    0.469181       -22.4483    -0.748625   -0.707328
  0.828013   -0.224381    0.573171       -12.7664     1.17309    -0.51876
 -0.76583    -0.32225    -0.595427        -7.43186   -0.810677   -0.242962
 -0.826892   -0.156963   -0.943456       -12.6295    -0.0534637  -0.989267
  0.0322462   0.0256327   1.15996     …  -11.0683     0.741776    0.519832
  0.143645    0.0254965  -0.913508       -16.8898    -0.411393    0.0781426
 -0.301467   -0.0380585   0.447166        10.1023     0.172388   -2.50261


In [17]:
max(abs.(C .- A)...)

4.547473508864641e-13

**mul!でA自身に解を格納させても、問題ない**

## まとめ
- 反復計算で何度も行列を更新する際にはmul!を使う
- コードの見た目の観点では A * inv(Diagonal(b)) の方が式が見えて良いので、状況に応じて使い分ける。

# 対角行列の逆行列計算は速いか？（まじめに逆行列計算してしまっていないか）

## 対角行列の逆行列計算

In [18]:
@benchmark inv(Diagonal(randn(10000)))

BenchmarkTools.Trial: 
  memory estimate:  156.44 KiB
  allocs estimate:  6
  --------------
  minimum time:     112.999 μs (0.00% GC)
  median time:      169.001 μs (0.00% GC)
  mean time:        178.725 μs (5.88% GC)
  maximum time:     8.199 ms (97.02% GC)
  --------------
  samples:          10000
  evals/sample:     1

## 密行列の逆行列計算

In [19]:
@benchmark inv(randn(10000,10000))

BenchmarkTools.Trial: 
  memory estimate:  1.49 GiB
  allocs estimate:  9
  --------------
  minimum time:     25.420 s (0.00% GC)
  median time:      25.420 s (0.00% GC)
  mean time:        25.420 s (0.00% GC)
  maximum time:     25.420 s (0.00% GC)
  --------------
  samples:          1
  evals/sample:     1

**対角行列の逆行列計算は速い**